
---

# 📘 Integration with LangSmith

[LangSmith](https://smith.langchain.com) helps monitor, debug, and visualize LangChain apps.
You can plug it into FastAPI endpoints to log and trace LLM runs.

---

## ✅ Step 1: Install Requirements

```bash
pip install fastapi uvicorn langchain langsmith openai
```

---

## ✅ Step 2: Setup Environment

Set your LangSmith and OpenAI credentials:

```bash
export LANGCHAIN_API_KEY=ls__your_langsmith_key
export LANGCHAIN_PROJECT=my-fastapi-app
export OPENAI_API_KEY=sk-xxxxxxx
```

Or use a `.env` file and load it with `python-dotenv`:

```bash
pip install python-dotenv
```

```env
# .env
LANGCHAIN_API_KEY=ls__your_langsmith_key
LANGCHAIN_PROJECT=my-fastapi-app
OPENAI_API_KEY=sk-xxxxxxx
```

---

## ✅ Step 3: Create LLM Chain with LangSmith Tracing

```python
# chains.py
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langsmith import traceable

llm = ChatOpenAI(model="gpt-3.5-turbo")

prompt = ChatPromptTemplate.from_template("Translate to French: {text}")
chain = LLMChain(llm=llm, prompt=prompt)

# Optional: Decorator to enable tracing
@traceable
def run_chain(input_text: str) -> str:
    return chain.run({"text": input_text})
```

---

## ✅ Step 4: Create FastAPI Server

```python
# main.py
from fastapi import FastAPI
from pydantic import BaseModel
from chains import run_chain

app = FastAPI()

class Input(BaseModel):
    text: str

@app.post("/translate/")
async def translate(input: Input):
    result = run_chain(input.text)
    return {"translated": result}
```

---

## ✅ Step 5: Run Server

```bash
uvicorn main:app --reload
```

Send a request:

```bash
curl -X POST http://127.0.0.1:8000/translate/ \
     -H "Content-Type: application/json" \
     -d '{"text": "Good morning"}'
```

---

## ✅ Step 6: View Traces in LangSmith

1. Go to [https://smith.langchain.com](https://smith.langchain.com)
2. Select your project: `my-fastapi-app`
3. Explore input/output, token usage, and debugging info

---

## ✅ Summary Table

| Component           | Description                      |
| ------------------- | -------------------------------- |
| `@traceable`        | Traces your chain in LangSmith   |
| `LANGCHAIN_PROJECT` | Groups logs by project name      |
| FastAPI Endpoint    | Sends user input to chain        |
| LangSmith           | Visualizes input, output, traces |

---

## 🧠 Optional Tips

* Attach `metadata` or `tags` to traces for filtering
* Use `tracer_v2 = LangChainTracer()` if doing advanced custom tracing
* Supports other LLMs (Anthropic, Mistral, etc.)

---
